In [18]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, date
import sys
import pickle as pk

In [42]:
# get the teams
# Teams = pd.read_csv('Teams.csv')
city_name_Dict = pk.load(open('city_name.pickle'))
name_pref_Tuples = pk.load(open('name_pref.pickle'))

In [44]:
# for regular season
BASE_URL = 'http://espn.go.com/nba/team/schedule/_/name/%s/year/%s/seasontype/2'

In [45]:
year = 2016

In [54]:
home_team = []
visit_team = []
home_score = []
visit_score = []
dates = []
match_id = []
# games = {}
for team, pref in name_pref_Tuples:
    url = BASE_URL % (pref, str(year))
    prev_year = year - 1
    html = urllib.urlopen(url).read()
    soup = BeautifulSoup(html, 'lxml')
    tables = soup.find_all('tr')[1:]
    # games[team] = {'rivals':[], 'score':[]}
    for table in tables:
        if table['class'][0] == 'colhead':
            if table.td.text == 'JANUARY':
                prev_year = year
            continue
        try:
            info = table.find_all('td')
            _d = datetime.strptime(str(prev_year) + ', ' + info[0].text, '%Y, %a, %b %d')
            _home = True if info[1].find(class_ = 'game-status').text == 'vs' else False
            try:
                _other_team = city_name_Dict[info[1].find(class_ = 'team-name').a.text]
            except Exception:
                if info[1].find(class_ = 'team-name').a.text == 'NY Knicks':
                    _other_team = 'New York Knicks'
                else:
                    _other_team = info[1].find(class_ = 'team-name').a.text
                pass
            
            _score = info[2].find(class_ = 'score').a.text.split('-')
            _won = True if info[2].li.span.text == 'W' else False
            _id = info[2].a['href'].split('/')[-1]
            
            home_team.append(team if _home else _other_team)
            visit_team.append(_other_team if _home else team)
            dates.append(_d)
            match_id.append(_id)
            
            
            if _home:
                if _won:
                    home_score.append(_score[0])
                    visit_score.append(_score[1])
                else:
                    home_score.append(_score[1])
                    visit_score.append(_score[0])
            else:
                if _won:
                    home_score.append(_score[1])
                    visit_score.append(_score[0])
                else:
                    home_score.append(_score[0])
                    visit_score.append(_score[1])
        except Exception as e:
            print e
#             print _d
            pass
#             sys.exit()

'NoneType' object has no attribute 'a'
'NoneType' object has no attribute 'a'
'NoneType' object has no attribute 'a'
'NoneType' object has no attribute 'a'


In [55]:
len(home_team)

2460

In [56]:
game_data = {'date':dates, 'match_id':match_id, 'home_team':home_team, 'visit_team':visit_team, 'home_score':home_score, 'visit_score':visit_score}

In [57]:
game_df = pd.DataFrame(game_data, columns = ['match_id', 'date', 'home_team', 'visit_team', 'home_score', 'visit_score']).drop_duplicates('match_id')

In [58]:
game_df2 = pd.DataFrame(data = game_df.values, index = range(len(game_df.index)), columns = ['match_id', 'date', 'home_team', 'visit_team', 'home_score', 'visit_score'])

In [59]:
game_df2

match_id                 date               home_team  \
0     400827892  2015-10-28 00:00:00          Boston Celtics   
1     400827911  2015-10-30 00:00:00          Boston Celtics   
2     400827927  2015-11-01 00:00:00          Boston Celtics   
3     400827947  2015-11-04 00:00:00          Indiana Pacers   
4     400827963  2015-11-06 00:00:00          Boston Celtics   
5     400827997  2015-11-10 00:00:00         Milwaukee Bucks   
6     400828003  2015-11-11 00:00:00          Boston Celtics   
7     400828016  2015-11-13 00:00:00          Boston Celtics   
8     400828036  2015-11-15 00:00:00   Oklahoma City Thunder   
9     400828041  2015-11-16 00:00:00         Houston Rockets   
10    400828055  2015-11-18 00:00:00          Boston Celtics   
11    400828066  2015-11-20 00:00:00          Boston Celtics   
12    400828084  2015-11-22 00:00:00           Brooklyn Nets   
13    400828097  2015-11-24 00:00:00           Atlanta Hawks   
14    400828104  2015-11-25 00:00:00          Boston Celtics   
15    400828116  2015-11-27 00:00:00          Boston Celtics   
16    400828136  2015-11-29 00:00:00           Orlando Magic   
17    400828142  2015-11-30 00:00:00              Miami Heat   
18    400828169  2015-12-03 00:00:00        Sacramento Kings   
19    400828183  2015-12-05 00:00:00       San Antonio Spurs   
20    400828199  2015-12-07 00:00:00    New Orleans Pelicans   
21    400828206  2015-12-09 00:00:00          Boston Celtics   
22    400828223  2015-12-11 00:00:00          Boston Celtics   
23    400828232  2015-12-12 00:00:00       Charlotte Hornets   
24    400828254  2015-12-15 00:00:00          Boston Celtics   
25    400828261  2015-12-16 00:00:00         Detroit Pistons   
26    400828276  2015-12-18 00:00:00          Boston Celtics   
27    400828298  2015-12-21 00:00:00          Boston Celtics   
28    400828310  2015-12-23 00:00:00       Charlotte Hornets   
29    400828333  2015-12-26 00:00:00         Detroit Pistons   
...         ...                  ...                     ...   
1200  400828417  2016-01-06 00:00:00  Minnesota Timberwolves   
1201  400828514  2016-01-19 00:00:00          Denver Nuggets   
1202  400828630  2016-02-03 00:00:00               Utah Jazz   
1203  400829047  2016-04-05 00:00:00          Denver Nuggets   
1204  400829085  2016-04-10 00:00:00          Denver Nuggets   
1205  400829117  2016-04-13 00:00:00  Portland Trail Blazers   
1206  400827937  2015-11-02 00:00:00  Minnesota Timberwolves   
1207  400828181  2015-12-05 00:00:00  Minnesota Timberwolves   
1208  400828366  2015-12-30 00:00:00  Minnesota Timberwolves   
1209  400828461  2016-01-12 00:00:00  Minnesota Timberwolves   
1210  400828481  2016-01-15 00:00:00   Oklahoma City Thunder   
1211  400828575  2016-01-27 00:00:00  Minnesota Timberwolves   
1212  400828591  2016-01-29 00:00:00               Utah Jazz   
1213  400828606  2016-01-31 00:00:00  Portland Trail Blazers   
1214  400828855  2016-03-11 00:00:00   Oklahoma City Thunder   
1215  400828974  2016-03-26 00:00:00  Minnesota Timberwolves   
1216  400829021  2016-04-01 00:00:00               Utah Jazz   
1217  400829079  2016-04-09 00:00:00  Portland Trail Blazers   
1218  400828095  2015-11-23 00:00:00               Utah Jazz   
1219  400828228  2015-12-11 00:00:00               Utah Jazz   
1220  400828243  2015-12-13 00:00:00   Oklahoma City Thunder   
1221  400828265  2015-12-16 00:00:00   Oklahoma City Thunder   
1222  400828451  2016-01-10 00:00:00  Portland Trail Blazers   
1223  400828880  2016-03-14 00:00:00   Oklahoma City Thunder   
1224  400828958  2016-03-24 00:00:00   Oklahoma City Thunder   
1225  400829058  2016-04-06 00:00:00  Portland Trail Blazers   
1226  400827955  2015-11-04 00:00:00               Utah Jazz   
1227  400828376  2015-12-31 00:00:00               Utah Jazz   
1228  400828471  2016-01-13 00:00:00  Portland Trail Blazers   
1229  400828719  2016-02-21 00:00:00  Portland Trail Blazers   

                  visit_team home_scor

In [60]:
game_df2.to_csv('Games.csv', index=False)